In [1]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
with open('input.txt', 'r') as f:
    text = f.read()

In [3]:
len(text)

1115394

In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


Tokenize

In [5]:
string_to_ind = {ch : i for i, ch in enumerate(chars)}
ind_to_string = {i : ch for i, ch in enumerate(chars)}

def encode(string):
    return [string_to_ind[ch] for ch in string]

def decode(ind):
    return ''.join(ind_to_string[i] for i in ind)

print(encode('hello'))
print(decode(encode('hello')))

[46, 43, 50, 50, 53]
hello


In [6]:
import torch
data = torch.tensor(encode(text), dtype=torch.long) ## here we tokenized the entire dataset in 

/Users/admin/Code/Personal/TrainingTransformers/.venv/lib/python3.13/site-packages/torch/_subclasses/functional_tensor.py:279: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


Train and Validation Split


In [7]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

We are now going to build datasets. We use a chunk size, to showcase how far are we going back to predict the next token. it also determines how far back are we looking. 

In [8]:
torch.manual_seed(1337)
block_size = 8
batch_size = 4

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [9]:
xb, yb = get_batch('train')
yb

tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])

Now we define our feed forward neural network

In [ ]:
import torch.nn as nn
from torch.nn import functional as F

class BLM(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        '''we are defining the table here helpful for inference'''
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # B -> batch, T -> time step or block size, C -> dimension of each token
        ## this predicts for each char, what should be the next char. 


    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) ## B, C
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            logits, loss = self(idx) ## this was B*T, C
            logits = logits[:, -1, :] # becomes (B, C), we only take the last predicted
            probs = F.softmax(logits, dim=1) # (B, C), softmax
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1), find the index
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1), add it back
        return idx


In [16]:
model = BLM(vocab_size)

out, loss = model(xb, yb)
print(out.shape)
print(loss)

torch.Size([32, 65])
tensor(4.5262, grad_fn=<NllLossBackward0>)


expectation of the log loss for a 65 class calssificatoin is -ln(1/65) ie, 4.7. this is if the output was random. 

In [22]:
idx = torch.zeros((1, 1), dtype=torch.long)
op = model.generate(idx, max_new_tokens=100)[0].tolist()
print(decode(op))


HaOtlq3P.,nh.$:y.yb,uIpmp,k:3;NE.gagavKmpoqtcBRMb&Njm?tkDuZN'aD:pFby.RjmM.FiHb$Qv
3q$pfZaUTTjPqSYIHX


In [23]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [ ]:
batch_size = 32

for steps in range(10000):

    xb, yb = get_batch('train')

    ##evaluate loss
    logits, loss = model(xb, yb)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4259066581726074


In [34]:
idx = torch.zeros((1, 1), dtype=torch.long)
op = model.generate(idx, max_new_tokens=1000)[0].tolist()
print(decode(op))



DUSond r ath n Morirvimout, dore y oly s tis sokillerseeciacesouen to he o his me t uch, TY ill wind whasuris thand
We sohainosat Shatedout weadurk, ANDINTENUD hadissure h
Han;

INDWinse
INat woulld: hisertitom, thearune.
Theices thil irt agrout.
HES:
Wh poraper thavice ty mat recen; onicr atambyour se s y we y ther ule he
LUK:
Buth

Toond d d, wainot G mehe;
A:
Crer seve me ayoug ran f thig y BRon thore ou fulo mu s se, h comeinoukithrenendowerd me ot folfith, yo
Thed whe w! bugon acong ' viur me

He
UCORLoroupy s acthe k,
D ne bsivante ive, d! lorshig, trwem wath f s h k TAno
Cirus hated thenillakek has telimyorns.
WAnthecl jeno.
Yon avor pom. womith ERUndend
on:
doun bendese beaverd wee hatheeras itht n,

Po st I gh,
I'd ce l p bush IXERCat:


Thuk l aro bayoucl che ce s t mouss, lond we ty:
SI'sothongort lr thotorind shingatonad idy'disorur REd:
RI:
LAUCANCathe'seathadubere athesorar mer EOLUSAn, oree, band bece.
IOH: andicthind amillemed ste and ting ma bu im d puso icharfan s p